# 1. Pandas

In [ ]:
import pandas as pd

Загрузим данные по указанной ссылке

In [ ]:
!wget -q http://files.grouplens.org/datasets/movielens/ml-100k.zip

Распакуем файл

In [ ]:
!unzip -q ml-100k.zip

Создадим датафреймы с фильмами и рейтингами

In [ ]:
ratings = pd.read_table('ml-100k/u.data', names=['user_id', 'movie_id', 'rating', 'timestamp'])

In [ ]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
column_names = ["movie_id", "movie_title", "release_date", "video_release_date", 
                "IMDb_URL", "unknown", "Action", "Adventure", "Animation", "Children's", 
                "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", 
                "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
movies = pd.read_table('ml-100k/u.item', delimiter='|', names=column_names, encoding='ISO-8859-1')

In [ ]:
movies.head()

,movie_id,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


Найдем ID пользователя, поставившего больше всего оценок

In [ ]:
ratings.groupby('user_id', ).count().sort_values('movie_id', ascending=False)

,movie_id,rating,timestamp
user_id,,,
405,737,737,737
655,685,685,685
13,636,636,636
450,540,540,540
276,518,518,518
...,...,...,...
685,20,20,20
475,20,20,20
36,20,20,20


Оставим в датафрейме ratings только те фильмы, который оценил данный пользователь 

In [ ]:
user405_ratings = ratings[ratings['user_id'] == 405]

In [ ]:
user405_ratings.head()

,user_id,movie_id,rating,timestamp
12276,405,56,4,885544911
12383,405,592,1,885548670
12430,405,1582,1,885548670
12449,405,171,1,885549544
12460,405,580,1,885547447


Признаки: общее количество оценок и суммарная оценка

In [ ]:
movies_ratings = ratings.groupby('movie_id')['rating'].agg(['count', 'sum']).reset_index()

In [ ]:
movies_ratings.rename(columns={'count': 'rating_count', 'sum': 'rating_sum'}, inplace=True)
movies_ratings.head()

,movie_id,rating_count,rating_sum
0,1,452,1753
1,2,131,420
2,3,90,273
3,4,209,742
4,5,86,284


Объединим датафреймы, добавим "жанры"

In [ ]:
df = pd.merge(left=user405_ratings, right=movies, on='movie_id')

In [ ]:
df.head()

,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,405,56,4,885544911,Pulp Fiction (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pulp%20Fictio...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,405,592,1,885548670,True Crime (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?True%20Crime%...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
2,405,1582,1,885548670,T-Men (1947),01-Jan-1947,NaN,http://us.imdb.com/M/title-exact?T-Men%20(1947),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,405,171,1,885549544,Delicatessen (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Delicatessen%...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
4,405,580,1,885547447,"Englishman Who Went Up a Hill, But Came Down a...",01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Englishman%20...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


Добавим количество оценок по фильмам, сумму оценок

In [ ]:
df = pd.merge(left=df, right=movies_ratings, on='movie_id')

In [ ]:
df.head()

,user_id,movie_id,rating,timestamp,movie_title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum
0,405,56,4,885544911,Pulp Fiction (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pulp%20Fictio...,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,394,1600
1,405,592,1,885548670,True Crime (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?True%20Crime%...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,9,30
2,405,1582,1,885548670,T-Men (1947),01-Jan-1947,NaN,http://us.imdb.com/M/title-exact?T-Men%20(1947),0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,405,171,1,885549544,Delicatessen (1991),01-Jan-1991,NaN,http://us.imdb.com/M/title-exact?Delicatessen%...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,65,252
4,405,580,1,885547447,"Englishman Who Went Up a Hill, But Came Down a...",01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Englishman%20...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,32,108


In [ ]:
df.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'movie_title',
       'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action',
       'Adventure', 'Animation', 'Children's', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'rating_count', 'rating_sum'],
      dtype='object')

In [ ]:
df.drop(columns=['movie_title', 'video_release_date', 'IMDb_URL'], inplace=True)

In [ ]:
df.head()

,user_id,movie_id,rating,timestamp,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum
0,405,56,4,885544911,01-Jan-1994,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,394,1600
1,405,592,1,885548670,01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,9,30
2,405,1582,1,885548670,01-Jan-1947,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,405,171,1,885549544,01-Jan-1991,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,65,252
4,405,580,1,885547447,01-Jan-1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,32,108


Выделим год из даты релиза и преобразуем его из object в числовой тип данных

In [ ]:
df['release_date'] = df['release_date'].str.split('-', expand=True)[2]

In [ ]:
df.head()

,user_id,movie_id,rating,timestamp,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum
0,405,56,4,885544911,1994,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,394,1600
1,405,592,1,885548670,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,9,30
2,405,1582,1,885548670,1947,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,405,171,1,885549544,1991,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,65,252
4,405,580,1,885547447,1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,32,108


In [ ]:
df.dtypes

user_id          int64
movie_id         int64
rating           int64
timestamp        int64
release_date    object
unknown          int64
Action           int64
Adventure        int64
Animation        int64
Children's       int64
Comedy           int64
Crime            int64
Documentary      int64
Drama            int64
Fantasy          int64
Film-Noir        int64
Horror           int64
Musical          int64
Mystery          int64
Romance          int64
Sci-Fi           int64
Thriller         int64
War              int64
Western          int64
rating_count     int64
rating_sum       int64
dtype: object

In [ ]:
df = df.astype({'release_date': 'int64'})

In [ ]:
df.dtypes

user_id         int64
movie_id        int64
rating          int64
timestamp       int64
release_date    int64
unknown         int64
Action          int64
Adventure       int64
Animation       int64
Children's      int64
Comedy          int64
Crime           int64
Documentary     int64
Drama           int64
Fantasy         int64
Film-Noir       int64
Horror          int64
Musical         int64
Mystery         int64
Romance         int64
Sci-Fi          int64
Thriller        int64
War             int64
Western         int64
rating_count    int64
rating_sum      int64
dtype: object

In [ ]:
df.head()

,user_id,movie_id,rating,timestamp,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum
0,405,56,4,885544911,1994,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,394,1600
1,405,592,1,885548670,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,9,30
2,405,1582,1,885548670,1947,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,405,171,1,885549544,1991,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,65,252
4,405,580,1,885547447,1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,32,108


Построим модель, предсказывающую оценку пользователя

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

Отделяем целевую часть

In [ ]:
X = df.drop('rating', axis=1)
y = df['rating']

In [ ]:
X.head()

,user_id,movie_id,timestamp,release_date,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating_count,rating_sum
0,405,56,885544911,1994,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,394,1600
1,405,592,885548670,1995,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,9,30
2,405,1582,885548670,1947,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1
3,405,171,885549544,1991,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,65,252
4,405,580,885547447,1995,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,32,108


In [ ]:
y.head()

0    4
1    1
2    1
3    1
4    1
Name: rating, dtype: int64

Делим данные на выборки для обучения и проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=11)

1. Линейная регрессия

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

1.386416714440769

2. Попробуем применить Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
mean_squared_error(y_test, rf.predict(X_test))

1.1944324324324322

Вывод: лучший результат дает применение модели Случайный лес

# 2. Spark

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark

In [ ]:
!pip install -q findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

Загрузим данные в Spark

In [ ]:
ratings_spark = spark.read.csv('ml-100k/u.data', inferSchema=True, header=False, sep='\t')

In [ ]:
ratings_spark.show()

+---+----+---+---------+
|_c0| _c1|_c2|      _c3|
+---+----+---+---------+
|196| 242|  3|881250949|
|186| 302|  3|891717742|
| 22| 377|  1|878887116|
|244|  51|  2|880606923|
|166| 346|  1|886397596|
|298| 474|  4|884182806|
|115| 265|  2|881171488|
|253| 465|  5|891628467|
|305| 451|  3|886324817|
|  6|  86|  3|883603013|
| 62| 257|  2|879372434|
|286|1014|  5|879781125|
|200| 222|  5|876042340|
|210|  40|  3|891035994|
|224|  29|  3|888104457|
|303| 785|  3|879485318|
|122| 387|  5|879270459|
|194| 274|  2|879539794|
|291|1042|  4|874834944|
|234|1184|  2|892079237|
+---+----+---+---------+
only showing top 20 rows



Переименуем колонки

In [ ]:
from functools import reduce

oldColumns = ratings_spark.schema.names
newColumns = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings_spark = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), ratings_spark)
ratings_spark.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [ ]:
movies_spark = spark.read.csv('ml-100k/u.item', inferSchema=True, header=False, sep='|', encoding='ISO-8859-1')

In [ ]:
movies_spark.show()

+---+--------------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|_c0|                 _c1|        _c2| _c3|                 _c4|_c5|_c6|_c7|_c8|_c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|
+---+--------------------+-----------+----+--------------------+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|  1|    Toy Story (1995)|01-Jan-1995|null|http://us.imdb.co...|  0|  0|  0|  1|  1|   1|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|
|  2|    GoldenEye (1995)|01-Jan-1995|null|http://us.imdb.co...|  0|  1|  1|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  3|   Four Rooms (1995)|01-Jan-1995|null|http://us.imdb.co...|  0|  0|  0|  0|  0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   0|   1|   0|   0|
|  4|   Get Shorty (1995)|01-Jan-1995|null|http://us.imdb.co...|  0|  

In [ ]:
oldColumns = movies_spark.schema.names
newColumns = ["movie_id", "movie_title", "release_date", "video_release_date", 
                "IMDb_URL", "unknown", "Action", "Adventure", "Animation", "Children's", 
                "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", 
                "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

movies_spark = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), movies_spark)
movies_spark.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|

Найдем среднюю оценку для каждого фильма

In [ ]:
avg_rating = ratings_spark.groupby('movie_id').avg('rating')
avg_rating.show()

+--------+------------------+
|movie_id|       avg(rating)|
+--------+------------------+
|     496| 4.121212121212121|
|     471|3.6108597285067874|
|     463| 3.859154929577465|
|     148|          3.203125|
|    1342|               2.5|
|     833| 3.204081632653061|
|    1088| 2.230769230769231|
|    1591|3.1666666666666665|
|    1238|             3.125|
|    1580|               1.0|
|    1645|               4.0|
|     392|3.5441176470588234|
|     623| 2.923076923076923|
|     540| 2.511627906976744|
|     858|               1.0|
|     737| 2.983050847457627|
|     243|2.4393939393939394|
|    1025|2.9318181818181817|
|    1084| 3.857142857142857|
|    1127| 2.909090909090909|
+--------+------------------+
only showing top 20 rows



Найдем среднюю оценку для каждого жанра

Добавим к датафрейму фильмов средние оценки

In [ ]:
new_df = movies_spark.join(avg_rating, on='movie_id', how='left_outer')
new_df.show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+------------------+
|movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|       avg(rating)|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+------------------+
|       1|    Toy Story (1995)| 01-Jan-1995|              null|http://us.imdb.co...|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|

Убедимся в отсутствии пустых значений в колонке avg(rating)

In [ ]:
new_df.describe().show()

+-------+------------------+--------------------+------------+------------------+--------------------+--------------------+-------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+
|summary|          movie_id|         movie_title|release_date|video_release_date|            IMDb_URL|             unknown|             Action|         Adventure|          Animation|         Children's|             Comedy|              Crime|         Documentary|              Drama|             Fantasy|           Film-Noir|              Horror|            Musical|            Mystery|            Romance|             Sci-Fi|          Thriller|                War|            Western|       avg

Создадим новый датафрем, в котором будут только жанры и средняя оценка по каждому фильму

In [ ]:
cols = ["unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
        "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
        "Romance", "Sci-Fi", "Thriller", "War", "Western", "avg(rating)"]

In [ ]:
genre_ratings = new_df.select(cols)

In [ ]:
genre_ratings.show()

+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+------------------+
|unknown|Action|Adventure|Animation|Children's|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|       avg(rating)|
+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+------------------+
|      0|     0|        0|        1|         1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|3.8783185840707963|
|      0|     1|        1|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       1|  0|      0|3.2061068702290076|
|      0|     0|        0|        0|         0|     0|    0|          0|    0|      0|        0|     0|      0|      0|      

In [ ]:
cols = ["unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
        "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
        "Romance", "Sci-Fi", "Thriller", "War", "Western"]

Создадим словарь: ключ - жанр, значение - средняя оценка

In [ ]:
genre_ratings_avg = {}

Пройдем по всем колонкам с жанрами, в которых выберем строки со значением, отличным от нуля, и посчитаем среднюю оценку

In [ ]:
for c in cols:
  temp_rating = genre_ratings.where((genre_ratings[c] != 0)).agg({'avg(rating)': 'avg'})
  genre_ratings_avg[c] = temp_rating.select('avg(avg(rating))').first()[0]

In [ ]:
genre_ratings_avg

{'Action': 2.966332403758986,
 'Adventure': 3.1439673136976105,
 'Animation': 3.2988130803823776,
 "Children's": 2.916884891868897,
 'Comedy': 3.0005649618406727,
 'Crime': 3.2110147495997547,
 'Documentary': 3.229273094093941,
 'Drama': 3.1873534142973314,
 'Fantasy': 2.849830297920943,
 'Film-Noir': 3.5483508757184237,
 'Horror': 2.730157352996138,
 'Musical': 3.376423123539873,
 'Mystery': 3.336813932383023,
 'Romance': 3.2440490301647995,
 'Sci-Fi': 3.1654460532512894,
 'Thriller': 3.136692475563836,
 'War': 3.489185428943569,
 'Western': 3.185617473056144,
 'unknown': 2.2222222222222223}

Получим датафрейм с пятью самыми популярными фильмами по количеству оценок

In [ ]:
rating_counts = ratings_spark.groupby('movie_id').count().orderBy('count', ascending=False)
best_movies = rating_counts.limit(5)

In [ ]:
best_movies = best_movies.join(movies_spark, on='movie_id', how='left_outer').select('movie_id', 'count', 'movie_title')

In [ ]:
best_movies.show()

+--------+-----+--------------------+
|movie_id|count|         movie_title|
+--------+-----+--------------------+
|      50|  583|    Star Wars (1977)|
|     258|  509|      Contact (1997)|
|     100|  508|        Fargo (1996)|
|     181|  507|Return of the Jed...|
|     294|  485|    Liar Liar (1997)|
+--------+-----+--------------------+



С пятью худшими фильмами

In [ ]:
rating_counts1 = ratings_spark.groupby('movie_id').count().orderBy('count', ascending=True)
worst_movies = rating_counts1.limit(5)

In [ ]:
worst_movies = worst_movies.join(movies_spark, on='movie_id', how='left_outer').select('movie_id', 'count', 'movie_title')

In [ ]:
worst_movies.show()

+--------+-----+--------------------+
|movie_id|count|         movie_title|
+--------+-----+--------------------+
|    1618|    1|King of New York ...|
|    1533|    1|I Don't Want to T...|
|    1339|    1|Stefano Quantesto...|
|    1460|    1|    Sleepover (1995)|
|    1650|    1|Butcher Boy, The ...|
+--------+-----+--------------------+

